In [17]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from mpl_toolkits.mplot3d import Axes3D
from sklearn.model_selection import train_test_split

# Carga el conjunto de datos desde un archivo CSV (reemplaza 'diabetes.csv' con tu nombre de archivo)
data = pd.read_csv('drive/MyDrive/diabetes/Diabetes Binary Classification.csv')

# Definir las características (X) y las etiquetas (y)
X = data.drop(columns=['Class variable (0 or 1)'])
y = data['Class variable (0 or 1)']

# Divide el conjunto de datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

df = data
labels = y

# Calcular la precisión en el conjunto de prueba
accuracy = accuracy_score(y_test, y_test)
print("Precisión en el conjunto de prueba:", accuracy)

# Mostrar el informe de clasificación en el conjunto de prueba
print("Informe de clasificación en el conjunto de prueba:")
print(classification_report(y_test, y_train))

# Calcular y mostrar la matriz de confusión en el conjunto de prueba
confusion = confusion_matrix(y_test, y_train)
print("Matriz de Confusión en el conjunto de prueba:")
print(confusion)

# Estandarizar y aplicar PCA para 3 componentes
scaler = StandardScaler()
data_rescaled = scaler.fit_transform(df)
pca = PCA(n_components=3)
principal_components = pca.fit_transform(data_rescaled)
principal_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2', 'PC3'])

# Entrenar regresión logística
logreg = LogisticRegression()
logreg.fit(principal_df, labels)

# Crear un grid tridimensional para evaluar el modelo
h = .1
x_min, x_max = principal_df['PC1'].min() - 1, principal_df['PC1'].max() + 1
y_min, y_max = principal_df['PC2'].min() - 1, principal_df['PC2'].max() + 1
z_min, z_max = principal_df['PC3'].min() - 1, principal_df['PC3'].max() + 1

xx, yy, zz = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h),
                         np.arange(z_min, z_max, h))
Z = logreg.predict(np.c_[xx.ravel(), yy.ravel(), zz.ravel()])
Z = Z.reshape(xx.shape)
# Visualizar en 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plotear la superficie de decisión
ax.scatter(principal_df['PC1'], principal_df['PC2'], principal_df['PC3'], c=labels, edgecolors='k', s=50, marker='o', depthshade=True)
ax.set_xlabel('Componente Principal 1')
ax.set_ylabel('Componente Principal 2')
ax.set_zlabel('Componente Principal 3')
plt.title('Decisión de Regresión Logística en espacio PCA 3D')

# Parámetros del hiperplano
w = logreg.coef_
b = logreg.intercept_

# Crear un grid para el hiperplano
x = np.linspace(principal_df['PC1'].min() - 1, principal_df['PC1'].max() + 1, 50)
y = np.linspace(principal_df['PC2'].min() - 1, principal_df['PC2'].max() + 1, 50)
X, Y = np.meshgrid(x, y)
Z = (-w[0][0] * X - w[0][1] * Y - b) / w[0][2]

# Visualizar en 3D
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plotear el hiperplano
ax.plot_surface(X, Y, Z, alpha=0.5, rstride=100, cstride=100)

# Plotear los puntos
ax.scatter(principal_df['PC1'], principal_df['PC2'], principal_df['PC3'], c=labels, edgecolors='k', s=50, marker='o', depthshade=True)

ax.set_xlabel('Componente Principal 1')
ax.set_ylabel('Componente Principal 2')
ax.set_zlabel('Componente Principal 3')
plt.title('Hiperplano separador en espacio PCA 3D')

plt.show()

Precisión en el conjunto de prueba: 1.0
Informe de clasificación en el conjunto de prueba:


ValueError: ignored